In [1]:
import dask
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from matplotlib import cm
import copy
import astropy
import hdbscan
import pandas as pd

from astropy.io import fits

In [2]:
# hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_clean.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)
# hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/redshift/rcsed_v2.fits')
hdul = fits.open('rcsed_v2_clean.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)

In [3]:
data = hdul[1].data

In [4]:
hdul.close()

In [5]:
DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder())

In [6]:
del data

In [ ]:
# DATA.to_csv('rcsed_v2_clean.csv')
# del DATA

In [7]:
DATA.shape

(4105516, 559)

In [65]:
(~DATA.z_sdss.isna()).sum()

3169675

In [8]:
DATA.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4105516 entries, 0 to 4105515
Data columns (total 559 columns):
ind                          4105516 non-null int32
ra                           4105516 non-null float64
dec                          4105516 non-null float64
z_sdss                       3169675 non-null float64
zerr_sdss                    3163704 non-null float64
specObjID_sdss               3171529 non-null float64
bestObjID_sdss               4105516 non-null int64
plate_sdss                   4105516 non-null int16
mjd_sdss                     4105516 non-null int32
fiberID_sdss                 4105516 non-null int16
z_2df                        220438 non-null float32
q_z_2df                      4105516 non-null int32
SeqNum_2df                   4105516 non-null object
z_6df                        116200 non-null float64
quality_6df                  4105516 non-null int16
targetname_6df               4105516 non-null object
specid_6df                   4105516 non

In [9]:
pd.set_option('max_columns', None)
DATA.head()

,ind,ra,dec,z_sdss,zerr_sdss,specObjID_sdss,bestObjID_sdss,plate_sdss,mjd_sdss,fiberID_sdss,z_2df,q_z_2df,SeqNum_2df,z_6df,quality_6df,targetname_6df,specid_6df,recno_uzc,z_uzc,z_err_uzc,z_cfa,z_err_cfa,z_hectospec,z_lamost,z_err_lamost,obsid_lamost,lmjd_lamost,mjd_lamost,planid_lamost,spid_lamost,fiberid_lamost,z_lega_c,f_z_lega_c,f_spec_lega_c,SPECT_ID_lega_c,Z_deep2,OBJNO_deep2,Z_deep3,OBJNO_deep3,z_wigglez,Q_wigglez,SpecFile_wigglez,Z_gama,NQ_gama,SPECID_gama,multiple_z,dz,E_bv,A_u_sdss,A_g_sdss,A_r_sdss,A_i_sdss,A_z_sdss,A_g_des,A_r_des,A_i_des,A_z_des,A_u_skymapper,A_v_skymapper,A_g_skymapper,A_r_skymapper,A_i_skymapper,A_z_skymapper,A_Y_ukidss,A_J_ukidss,A_H_ukidss,A_K_ukidss,A_Z_viking,A_Y_viking,A_J_viking,A_H_viking,A_ks_viking,A_g_panstarrs,A_r_panstarrs,A_i_pansatrrs,A_z_panstarrs,A_y_panstarrs,A_w1_wise,A_w2_wise,A_w3_wise,A_w4_wise,A_NUV_galex,A_FUV_galex,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,fibermag_u_sdss,fibermag_g_sdss,fibermag_r_sdss,fibermag_i_sdss,fibermag_z_sdss,fibermagerr_u_sdss,fibermagerr_g_sdss,fibermagerr_r_sdss,fibermagerr_i_sdss,fibermagerr_z_sdss,fiber2mag_u_sdss,fiber2mag_g_sdss,fiber2mag_r_sdss,fiber2mag_i_sdss,fiber2mag_z_sdss,fiber2magerr_u_sdss,fiber2magerr_g_sdss,fiber2magerr_r_sdss,fiber2magerr_i_sdss,fiber2magerr_z_sdss,petromag_u_sdss,petromag_g_sdss,petromag_r_sdss,petromag_i_sdss,petromag_z_sdss,petromagerr_u_sdss,petromagerr_g_sdss,petromagerr_r_sdss,petromagerr_i_sdss,petromagerr_z_sdss,petror50_u_sdss,petror50_g_sdss,petror50_r_sdss,petror50_i_sdss,petror50_z_sdss,magerr_petro_g_des,magerr_petro_r_des,magerr_petro_i_des,magerr_petro_z_des,magerr_petro_y_des,mag_petro_g_des,mag_petro_r_des,mag_petro_i_des,mag_petro_z_des,mag_petro_y_des,petro_radius_g_des,petro_radius_r_des,petro_radius_i_des,petro_radius_z_des,petro_radius_y_des,mag_aper_8_g_des,magerr_aper_8_g_des,mag_aper_8_r_des,magerr_aper_8_r_des,mag_aper_8_i_des,magerr_aper_8_i_des,mag_aper_8_z_des,magerr_aper_8_z_des,mag_aper_8_y_des,magerr_aper_8_y_des,mag_auto_g_des,magerr_auto_g_des,mag_auto_r_des,magerr_auto_r_des,mag_auto_i_des,magerr_auto_i_des,mag_auto_z_des,magerr_auto_z_des,mag_auto_y_des,magerr_auto_y_des,mag_aper_8_g_dered,mag_aper_8_r_dered,mag_aper_8_i_dered,mag_aper_8_z_dered,mag_aper_8_y_dered,mag_auto_g_dered,mag_auto_r_dered,mag_auto_i_dered,mag_auto_z_dered,mag_auto_y_dered,kron_radius_g_des,kron_radius_r_des,kron_radius_i_des,kron_radius_z_des,kron_radius_y_des,umag_kids,e_umag_kids,gmag_kids,e_gmag_kids,rmag_kids,e_rmag_kids,imag_kids,e_imag_kids,u_petro_skymapper,e_u_petro_skymapper,v_petro_skymapper,e_v_petro_skymapper,g_petro_skymapper,e_g_petro_skymapper,r_petro_skymapper,e_r_petro_skymapper,i_petro_skymapper,e_i_petro_skymapper,z_petro_skymapper,e_z_petro_skymapper,JPETROMAG_uhs,JPETROMAGERR_uhs,JAPERMAG3_uhs,JAPERMAG3ERR_uhs,JAPERMAG4_uhs,JAPERMAG4ERR_uhs,JAPERMAG6_uhs,JAPERMAG6ERR_uhs,YPETROMAG_ukidss,YPETROMAGERR_ukidss,YAPERMAG3_ukidss,YAPERMAG3ERR_ukidss,YAPERMAG4_ukidss,YAPERMAG4ERR_ukidss,YAPERMAG6_ukidss,YAPERMAG6ERR_ukidss,JPETROMAG_ukidss,JPETROMAGERR_ukidss,JAPERMAG3_ukidss,JAPERMAG3ERR_ukidss,JAPERMAG4_ukidss,JAPERMAG4ERR_ukidss,JAPERMAG6_ukidss,JAPERMAG6ERR_ukidss,J_1PETROMAG_ukidss,J_1PETROMAGERR_ukidss,J_1APERMAG3_ukidss,J_1APERMAG3ERR_ukidss,J_1APERMAG4_ukidss,J_1APERMAG4ERR_ukidss,J_1APERMAG6_ukidss,J_1APERMAG6ERR_ukidss,J_2PETROMAG_ukidss,J_2PETROMAGERR_ukidss,J_2APERMAG3_ukidss,J_2APERMAG3ERR_ukidss,J_2APERMAG4_ukidss,J_2APERMAG4ERR_ukidss,J_2APERMAG6_ukidss,J_2APERMAG6ERR_ukidss,HPETROMAG_ukidss,HPETROMAGERR_ukidss,HAPERMAG3_ukidss,HAPERMAG3ERR_ukidss,HAPERMAG4_ukidss,HAPERMAG4ERR_ukidss,HAPERMAG6_ukidss,HAPERMAG6ERR_ukidss,KPETROMAG_ukidss,KPETROMAGERR_ukidss,KPSFMAG_ukidss,KPSFMAGERR_ukidss,KSERMAG2D_ukidss,KSERMAG2DERR_ukidss,KAPERMAG3_ukidss,KAPERMAG3ERR_ukidss,KAPERMAG4_ukidss,KAPERMAG4ERR_ukidss,KAPERMAG6_ukidss,KAPERMAG6ERR_ukidss,EBV_ukidss,AY_ukidss,AH_ukidss,AK_ukidss,yPetroMag_vhs,yPetroMagErr_vhs,yAperMag3_vhs,yAperMag3Err_vhs,yAperMag4_vhs,y

**Наблюдения:**
- в некоторых колонках,  в которых присутствуют все значния есть очень странные значения вроде "-2147483648" или "-32768" и другие

In [83]:
#    Здесь мы создадим датафрейм с наиболее 
#       частотными значениями по каждому из столбцов

top_values = []

for col in DATA.columns[1:]:
    vc = DATA[col].value_counts(ascending=False).head(1)
    top_values.append([vc.name, vc.index[0], vc[vc.index[0]]])
    
anomaly = pd.DataFrame(data=top_values, columns = ['column_name', 'anomaly_value', 'value_counts'])

In [84]:
# Здесь мы создаем колонку с количеством
#  ненулевых значений в каждом из столбцов таблицы
#     DATA, начиная с ra

not_nan = []

for col in DATA.columns[1:]:
    not_nan.append((~DATA[col].isna()).sum())
    
not_nan = pd.Series(not_nan)

In [85]:
# Дополним получившийся датафрейм столбцом, показывающим,
# какую долю от длины всего столбца занимает каждое аномальное значение

import copy

top_values_data = copy.copy(anomaly)

top_values_data['percent_total'] = (top_values_data.value_counts / len(DATA)).round(3)
top_values_data['percent_not_nan'] = (top_values_data.value_counts / not_nan).round(3)

In [86]:
# Выведем на экран получившйся датафрейм

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='percent_not_nan', ascending=False))

,column_name,anomaly_value,value_counts,percent_total,percent_not_nan
18,z_err_uzc,0,19372,0.005,1.000
33,SPECT_ID_lega_c,b' ',4103699,1.000,1.000
32,f_spec_lega_c,-32768,4103699,1.000,1.000
245,KSERMAG2DERR_ukidss,-9.99999e+08,1070222,0.261,1.000
244,KSERMAG2D_ukidss,-9.99999e+08,1070222,0.261,1.000
243,KPSFMAGERR_ukidss,-9.99999e+08,1070222,0.261,1.000
242,KPSFMAG_ukidss,-9.99999e+08,1070222,0.261,1.000
31,f_z_lega_c,-32768,4103699,1.000,1.000
20,z_err_cfa,0,40726,0.010,1.000
37,OBJNO_deep3,-2147483648,4098306,0.998,0.998


### Анализ данных на основе таблицы top_values_data

Таблица **top_values_data** содержит в себе следующие столбцы:
- **column_name**   -   названия столбцов таблицы **DATA**

- **anomaly_value**   -   наиболее частотные значения столбцов таблицы **DATA**

- **value_counts**   -   количество раз, которое *anomaly_value* встречается в столбце таблицы **DATA**

- **percent_total**   -   доля *anomaly_value* от всего объема столбца таблицы **DATA**

- **percent_not_nan**   -   доля *anomaly_value* от non-null объема столбца таблицы **DATA**

В ячейке ниже показано, сколько столбцов таблицы **DATA** содержат более **90% и т.д.** аномальных значений от общего количества non-null значений столбца. Оказывается, что около трети столбцов (162) содержат таки аномалий больше, чем **10%**.

**! Нужно понять, какой порог считать допустимым.**

**! Остальные можно отбросить.**


In [113]:
for num in np.arange(0.9,0.0,-0.05).round(2):
    print(num, '   ', len(top_values_data.query('percent_not_nan > @num')))

0.9     32
0.85     40
0.8     40
0.75     40
0.7     42
0.65     71
0.6     71
0.55     88
0.5     88
0.45     93
0.4     101
0.35     101
0.3     103
0.25     108
0.2     114
0.15     117
0.1     162
0.05     219


In [116]:
#     Добавим столбец non_null_part, показываающий 
#     какую часть от величины столбца составляют ненулевые значения

top_values_data['non_null_part'] = (not_nan / len(DATA)).round(3)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='non_null_part', ascending=True))

,column_name,anomaly_value,value_counts,percent_total,percent_not_nan,non_null_part
30,z_lega_c,0.7282,5,0.000,0.003,0.000
180,u_petro_skymapper,18.287,17,0.000,0.002,0.002
182,v_petro_skymapper,17.637,11,0.000,0.002,0.002
183,e_v_petro_skymapper,0.134,48,0.000,0.007,0.002
181,e_u_petro_skymapper,0.147,90,0.000,0.009,0.002
36,Z_deep3,0.533413,4,0.000,0.001,0.002
17,z_uzc,0.0171952,9,0.000,0.000,0.005
18,z_err_uzc,0,19372,0.005,1.000,0.005
184,g_petro_skymapper,18.139,31,0.000,0.001,0.008
34,Z_deep2,1.24116,3,0.000,0.000,0.008


In [119]:
# Выясним сколько столбцов таблицы DATA  заполнены меньше чем на num*100 %

for num in np.arange(0.9,0.0,-0.1).round(2):
    print(num, '   ', len(top_values_data.query('non_null_part < @num')))

0.9     479
0.8     419
0.7     414
0.6     414
0.5     373
0.4     301
0.3     258
0.2     202
0.1     120


In [120]:
len(top_values_data.query('(non_null_part<0.01)&(percent_not_nan<0.5)'))

10

In [114]:
# Выведем на экран получившйся датафрейм

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='value_counts', ascending=False))

,column_name,anomaly_value,value_counts,percent_total,percent_not_nan
33,SPECT_ID_lega_c,b' ',4103699,1.000,1.000
31,f_z_lega_c,-32768,4103699,1.000,1.000
32,f_spec_lega_c,-32768,4103699,1.000,1.000
37,OBJNO_deep3,-2147483648,4098306,0.998,0.998
16,recno_uzc,-2147483648,4086037,0.995,0.995
45,dz,70,4080327,0.994,0.994
514,phot_data_new_galex,84,4076694,0.993,0.993
35,OBJNO_deep2,-2147483648,4071053,0.992,0.992
15,specid_6df,-2147483648,3989227,0.972,0.972
14,targetname_6df,b' ',3989227,0.972,0.972


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(DATA.describe(include='object'))

,SeqNum_2df,targetname_6df,planid_lamost,SPECT_ID_lega_c,SpecFile_wigglez,SPECID_gama
count,4105516,4105516,4105516,4105516,4105516,4105516
unique,222290,115740,3143,1796,223883,158365
top,b' ',b' ',b' ',b' ',b' ',b' '
freq,3882161,3989227,3880087,4103699,3881571,3946981


In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(DATA.describe())

,ind,ra,dec,z_sdss,zerr_sdss,specObjID_sdss,bestObjID_sdss,plate_sdss,mjd_sdss,fiberID_sdss,z_2df,q_z_2df,z_6df,quality_6df,specid_6df,recno_uzc,z_uzc,z_err_uzc,z_cfa,z_err_cfa,z_hectospec,z_lamost,z_err_lamost,obsid_lamost,lmjd_lamost,mjd_lamost,spid_lamost,fiberid_lamost,z_lega_c,f_z_lega_c,f_spec_lega_c,Z_deep2,OBJNO_deep2,Z_deep3,OBJNO_deep3,z_wigglez,Q_wigglez,Z_gama,NQ_gama,multiple_z,dz,E_bv,A_u_sdss,A_g_sdss,A_r_sdss,A_i_sdss,A_z_sdss,A_g_des,A_r_des,A_i_des,A_z_des,A_u_skymapper,A_v_skymapper,A_g_skymapper,A_r_skymapper,A_i_skymapper,A_z_skymapper,A_Y_ukidss,A_J_ukidss,A_H_ukidss,A_K_ukidss,A_Z_viking,A_Y_viking,A_J_viking,A_H_viking,A_ks_viking,A_g_panstarrs,A_r_panstarrs,A_i_pansatrrs,A_z_panstarrs,A_y_panstarrs,A_w1_wise,A_w2_wise,A_w3_wise,A_w4_wise,A_NUV_galex,A_FUV_galex,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,fibermag_u_sdss,fibermag_g_sdss,fibermag_r_sdss,fibermag_i_sdss,fibermag_z_sdss,fibermagerr_u_sdss,fibermagerr_g_sdss,fibermagerr_r_sdss,fibermagerr_i_sdss,fibermagerr_z_sdss,fiber2mag_u_sdss,fiber2mag_g_sdss,fiber2mag_r_sdss,fiber2mag_i_sdss,fiber2mag_z_sdss,fiber2magerr_u_sdss,fiber2magerr_g_sdss,fiber2magerr_r_sdss,fiber2magerr_i_sdss,fiber2magerr_z_sdss,petromag_u_sdss,petromag_g_sdss,petromag_r_sdss,petromag_i_sdss,petromag_z_sdss,petromagerr_u_sdss,petromagerr_g_sdss,petromagerr_r_sdss,petromagerr_i_sdss,petromagerr_z_sdss,petror50_u_sdss,petror50_g_sdss,petror50_r_sdss,petror50_i_sdss,petror50_z_sdss,magerr_petro_g_des,magerr_petro_r_des,magerr_petro_i_des,magerr_petro_z_des,magerr_petro_y_des,mag_petro_g_des,mag_petro_r_des,mag_petro_i_des,mag_petro_z_des,mag_petro_y_des,petro_radius_g_des,petro_radius_r_des,petro_radius_i_des,petro_radius_z_des,petro_radius_y_des,mag_aper_8_g_des,magerr_aper_8_g_des,mag_aper_8_r_des,magerr_aper_8_r_des,mag_aper_8_i_des,magerr_aper_8_i_des,mag_aper_8_z_des,magerr_aper_8_z_des,mag_aper_8_y_des,magerr_aper_8_y_des,mag_auto_g_des,magerr_auto_g_des,mag_auto_r_des,magerr_auto_r_des,mag_auto_i_des,magerr_auto_i_des,mag_auto_z_des,magerr_auto_z_des,mag_auto_y_des,magerr_auto_y_des,mag_aper_8_g_dered,mag_aper_8_r_dered,mag_aper_8_i_dered,mag_aper_8_z_dered,mag_aper_8_y_dered,mag_auto_g_dered,mag_auto_r_dered,mag_auto_i_dered,mag_auto_z_dered,mag_auto_y_dered,kron_radius_g_des,kron_radius_r_des,kron_radius_i_des,kron_radius_z_des,kron_radius_y_des,umag_kids,e_umag_kids,gmag_kids,e_gmag_kids,rmag_kids,e_rmag_kids,imag_kids,e_imag_kids,u_petro_skymapper,e_u_petro_skymapper,v_petro_skymapper,e_v_petro_skymapper,g_petro_skymapper,e_g_petro_skymapper,r_petro_skymapper,e_r_petro_skymapper,i_petro_skymapper,e_i_petro_skymapper,z_petro_skymapper,e_z_petro_skymapper,JPETROMAG_uhs,JPETROMAGERR_uhs,JAPERMAG3_uhs,JAPERMAG3ERR_uhs,JAPERMAG4_uhs,JAPERMAG4ERR_uhs,JAPERMAG6_uhs,JAPERMAG6ERR_uhs,YPETROMAG_ukidss,YPETROMAGERR_ukidss,YAPERMAG3_ukidss,YAPERMAG3ERR_ukidss,YAPERMAG4_ukidss,YAPERMAG4ERR_ukidss,YAPERMAG6_ukidss,YAPERMAG6ERR_ukidss,JPETROMAG_ukidss,JPETROMAGERR_ukidss,JAPERMAG3_ukidss,JAPERMAG3ERR_ukidss,JAPERMAG4_ukidss,JAPERMAG4ERR_ukidss,JAPERMAG6_ukidss,JAPERMAG6ERR_ukidss,J_1PETROMAG_ukidss,J_1PETROMAGERR_ukidss,J_1APERMAG3_ukidss,J_1APERMAG3ERR_ukidss,J_1APERMAG4_ukidss,J_1APERMAG4ERR_ukidss,J_1APERMAG6_ukidss,J_1APERMAG6ERR_ukidss,J_2PETROMAG_ukidss,J_2PETROMAGERR_ukidss,J_2APERMAG3_ukidss,J_2APERMAG3ERR_ukidss,J_2APERMAG4_ukidss,J_2APERMAG4ERR_ukidss,J_2APERMAG6_ukidss,J_2APERMAG6ERR_ukidss,HPETROMAG_ukidss,HPETROMAGERR_ukidss,HAPERMAG3_ukidss,HAPERMAG3ERR_ukidss,HAPERMAG4_ukidss,HAPERMAG4ERR_ukidss,HAPERMAG6_ukidss,HAPERMAG6ERR_ukidss,KPETROMAG_ukidss,KPETROMAGERR_ukidss,KPSFMAG_ukidss,KPSFMAGERR_ukidss,KSERMAG2D_ukidss,KSERMAG2DERR_ukidss,KAPERMAG3_ukidss,KAPERMAG3ERR_ukidss,KAPERMAG4_ukidss,KAPERMAG4ERR_ukidss,KAPERMAG6_ukidss,KAPERMAG6ERR_ukidss,EBV_ukidss,AY_ukidss,AH_ukidss,AK_ukidss,yPetroMag_vhs,yPetroMagErr_vhs,yAperMag3_vhs,yAperMag3Err_vhs,yAperMag4_vhs,yAperMag4Err_vhs,yAperMag6_vhs,yAperMag6Err_vhs,jPetroMag_vhs,jPetroMagErr_vhs,jAperMa